# Memoria de Corto Plazo con Elasticsearch

En este notebook implementamos **memoria de corto plazo (short-term memory)** usando **Elasticsearch** como backend de almacenamiento.

### Ventajas de usar Elasticsearch:

- Búsqueda distribuida y escalable
- Persistencia duradera entre reinicios
- Capacidades de búsqueda avanzada sobre el historial
- Ideal para entornos de producción

*Requisito para esta POC**: Elasticsearch debe estar corriendo en `localhost:9200`


### 1. Imports y Configuración Inicial


In [13]:

import sys
import importlib
import dotenv
from agents import Agent, Runner, trace
dotenv.load_dotenv()

True

In [14]:
# Recargar el módulo elasticsearch_session si ya estaba cargado
# (útil durante desarrollo para reflejar cambios sin reiniciar el kernel)
if 'elasticsearch_session' in sys.modules:
    import elasticsearch_session
    importlib.reload(elasticsearch_session)
    print("Módulo elasticsearch_session recargado")
else:
    print("------  Primera carga del módulo")

# Importar nuestra implementación personalizada de sesión con Elasticsearch
from elasticsearch_session import ElasticsearchSession

Módulo elasticsearch_session recargado


### 2. Definición del Agente


In [15]:
nutrition_agent = Agent(
    name="Asistente Nutricional",
    instructions="""
    Eres un asistente que compara qué tan saludables son diferentes alimentos.
    Cuando respondas, da una lista de qué tan saludables son los alimentos con una puntuación del 1 al 10.
    Ordena por: el alimento más saludable va primero.

    Ejemplo:
    Pregunta: Compara X y Y
    Respuesta: X es más saludable que Y.
    1) X: 8/10 - Muy saludable pero alto en fructosa
    2) Y: 3/10 - Alto en azúcar y grasa
    """,
)


### 3. Demostración: Sin Memoria

Primero probamos el agente **sin memoria**. 

In [17]:
# Primera consulta sin memoria
result = await Runner.run(nutrition_agent, "¿Qué es más saludable, bananas o chupetín?")
print(result.final_output)


1) Bananas: 8/10 - Muy saludable; ricas en fibra y potasio, vitaminas. Azúcar natural.
2) Chupetín: 2/10 - Alto en azúcar y calorías, sin nutrientes beneficios.


In [19]:
# Segunda consulta sin memoria - El agente NO recordará la conversación anterior
result = await Runner.run(nutrition_agent, "Agrega manzanas a la comparación")
print(result.final_output)


Para añadir las manzanas necesito saber con qué otros alimentos las quieres comparar. ¿Qué alimentos quieres comparar junto con las manzanas?


El agente no sabe a qué comparación nos referimos porque no tiene memoria de la conversación anterior.


### 4. Configurar Memoria con Elasticsearch

Ahora vamos a crear una **sesión persistente** usando Elasticsearch.


#### 4.1. Limpiar índice anterior (opcional)

Si ejecutaste este notebook antes, es recomendable limpiar el índice para empezar con una sesión nueva:


In [20]:
from elasticsearch import AsyncElasticsearch

# Conectar temporalmente para limpiar índice anterior
temp_client = AsyncElasticsearch(hosts=["http://localhost:9200"])

index_name = "agent_sessions"
if await temp_client.indices.exists(index=index_name):
    await temp_client.indices.delete(index=index_name)
    print(f"Indice '{index_name}' eliminado")
else:
    print(f"Indice '{index_name}' no existe (se creará automáticamente)")

await temp_client.close()


Indice 'agent_sessions' eliminado


### 4.2. Crear sesión de Elasticsearch


In [21]:
# Crear sesión de Elasticsearch
# El índice se creará automáticamente con la estructura correcta
session = ElasticsearchSession(
    session_id="conversation_nutrition_demo",
    es_host="localhost:9200",
    index_name="agent_sessions"
)

print(f"Sesión creada con session_id: {session.session_id}")


Sesión creada con session_id: conversation_nutrition_demo


### 5. Ahora con Memoria de contexto en Elastic

Ahora el agente **recordará** el contexto de la conversación.


In [22]:
# Cuando ejecutamos el agente agregamos la sesión
result = await Runner.run(
    nutrition_agent, 
    "¿Qué es más saludable, bananas o churros?",
    session=session
)
print(result.final_output)


[DEBUG] add_items() llamado con 1 items
[DEBUG] Última secuencia: 0
[DEBUG] Item 0: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=87
[DEBUG] Ejecutando bulk insert con 1 documentos
[DEBUG] Bulk insert exitoso: errors=False
[DEBUG] add_items() llamado con 2 items
[DEBUG] Última secuencia: 1
[DEBUG] Item 0: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=99
[DEBUG] Item 1: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=434
[DEBUG] Ejecutando bulk insert con 2 documentos
[DEBUG] Bulk insert exitoso: errors=False
Bananas son más saludables que churros.
1) Bananas: 8/10 - Nutritivas, ricas en fibra y potasio; bajo contenido de grasa.
2) Churros: 3/10 - Alto en azúcar, grasas y calorías; fritos y con harina refinada.


In [23]:
# Segunda consulta CON memoria - debe tirar de Elasticsearch

with trace("Asistente Nutricional con Memoria Elasticsearch"):
    result = await Runner.run(
        nutrition_agent, 
        "Agrega manzanas a la comparación",
        session=session
    )

print(result.final_output)


[DEBUG] add_items() llamado con 1 items
[DEBUG] Última secuencia: 3
[DEBUG] Item 0: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=68
[DEBUG] Ejecutando bulk insert con 1 documentos
[DEBUG] Bulk insert exitoso: errors=False
[DEBUG] add_items() llamado con 2 items
[DEBUG] Última secuencia: 4
[DEBUG] Item 0: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=99
[DEBUG] Item 1: tipo=<class 'dict'>
[DEBUG] Item serializado exitosamente, longitud=569
[DEBUG] Ejecutando bulk insert con 2 documentos
[DEBUG] Bulk insert exitoso: errors=False
Bananas es más saludable que las manzanas, y las dos son más saludables que los churros.

1) Bananas: 8/10 - Nutritivas, ricas en fibra y potasio; bajo contenido de grasa.
2) Manzanas: 7/10 - Buena fibra y vitamina C; azúcares naturales y menos potasio que la banana.
3) Churros: 3/10 - Alto en azúcar, grasa y calorías; fritos.


### 6. Verificación del Historial

Podemos inspeccionar el historial almacenado en Elasticsearch:


In [25]:
# Obtener historial usando el método de la sesión
items = await session.get_items()
print(f"Total de mensajes en el historial: {len(items)}")
print(f"\nPrimeros 5 mensajes:")

for i, item in enumerate(items[:5]):
    print(f"\n--- Mensaje {i+1} ---")
    if isinstance(item, dict):
        role = item.get('role', 'unknown')
        print(f"Rol: {role}")
        
        # Mostrar contenido si es un mensaje de usuario
        if 'content' in item and isinstance(item['content'], list):
            for content_part in item['content']:
                if isinstance(content_part, dict) and 'text' in content_part:
                    text = content_part['text']
                    # Truncar si es muy largo
                    if len(text) > 150:
                        text = text[:150] + "..."
                    print(f"Texto: {text}")


Total de mensajes en el historial: 6

Primeros 5 mensajes:

--- Mensaje 1 ---
Rol: user

--- Mensaje 2 ---
Rol: unknown

--- Mensaje 3 ---
Rol: assistant
Texto: Bananas son más saludables que churros.
1) Bananas: 8/10 - Nutritivas, ricas en fibra y potasio; bajo contenido de grasa.
2) Churros: 3/10 - Alto en a...

--- Mensaje 4 ---
Rol: user

--- Mensaje 5 ---
Rol: unknown
